In [1]:
import numpy as np
import os

## Load the data
data = np.load("../input/practice/224_data.npy")
labels = np.load("../input/practice/224_labels.npy")

print(data.shape)
print(labels.shape)

(27558, 224, 224, 3)
(27558, 2)


In [2]:
from sklearn.model_selection import train_test_split

## Train Test Split

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)
print(X_train.shape)
print(X_test.shape)

(22046, 224, 224, 3)
(5512, 224, 224, 3)


In [3]:
from keras.applications.vgg19 import VGG19
from keras.layers import Dense, Dropout, GlobalAveragePooling2D
from keras.models import Model


vgg_19 = VGG19(weights='imagenet',include_top=False,input_shape=(224,224,3))
vgg_19 = Model(input=vgg_19.input,output=vgg_19.get_layer('block5_conv3').output)
#vgg_19.summary()

x = vgg_19.output
x = GlobalAveragePooling2D()(x)
x= Dense(1024,activation='relu')(x)
x = Dropout(0.5)(x)
prediction = Dense(2, activation='softmax', name='predictions')(x)

from keras.optimizers import SGD

model = Model(inputs=vgg_19.input,outputs=prediction)

for layer in vgg_19.layers:
    layer.trainable=False

sgd = SGD(lr=0.0001, decay=1e-6, momentum=0.9, nesterov=True) 

model.compile(optimizer=sgd,
              loss='mse',
              metrics=['accuracy'])

model.fit(X_train, y_train,
      batch_size=32,
      epochs=60,
      shuffle=True,
      validation_split=0.1,
      verbose=1)

Using TensorFlow backend.


80142336/80134624 [==============================] - 9s 0us/step


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("bl...)`
  


Train on 19841 samples, validate on 2205 samples
Epoch 1/60
 4256/19841 [=====>........................] - ETA: 1:03 - loss: 0.5035 - acc: 0.4918

KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score


y_pred = model.predict(X_test)


print("Accuracy: ",accuracy_score(y_test.argmax(axis=1),y_pred.argmax(axis=1)))
print("F1_score: ",f1_score(y_test.argmax(axis=1),y_pred.argmax(axis=1)))
print("Recall: ",recall_score(y_test.argmax(axis=1),y_pred.argmax(axis=1)))
print("Confusion Matrix",confusion_matrix(y_test.argmax(axis=1),y_pred.argmax(axis=1)))